In [76]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim as gyNominatim
from pgeocode import Nominatim as pgNominatim #Easier in locating latitude & longitude using postal code
import folium

# Part 1: Building the Neighborhoods Data Frame

In [140]:
#Get Data and populate data frame
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs1 = pd.read_html(url, attrs={'class': 'wikitable'})
Neighborhoods_df = pd.DataFrame(dfs1[0])

In [141]:
#Ignore cells with a borough that is Not assigned
Neighborhoods_df = Neighborhoods_df[Neighborhoods_df.Borough != 'Not assigned'].reset_index(drop=True)

In [142]:
#Combined neighborhoods with same postal code into one row separated with a comma
Neighborhoods_df['Neighborhood'] = Neighborhoods_df['Neighborhood'].str.replace(' /', ',')

In [143]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
Neighborhoods_df['Neighborhood'] = pd.np.where(Neighborhoods_df.Neighborhood.str.contains("Not assigned"), Neighborhoods_df.Borough, Neighborhoods_df.Neighborhood)

In [144]:
Neighborhoods_df.head(11)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [145]:
Neighborhoods_df.shape

(103, 3)

# Part 2: Add latitude and longitude to the Data Frame

In [148]:
#Get saved data for better latitude and longitude accuracy
lat_long = pd.read_csv('https://cocl.us/Geospatial_data')

In [ ]:
#Add latitude and longitude to dataframe
Neighborhoods_df = Neighborhoods_df.join(lat_long.set_index('Postal Code'), on='Postal code')

In [126]:
#Build Geolocator for Canada
#geolocator = pgNominatim('CA')
#Get Latitude and Longitude
#Neighborhoods_df['Latitude'] = Neighborhoods_df['Postal code'].apply(geolocator.query_postal_code)[["latitude"]]
#Neighborhoods_df['Longitude'] = Neighborhoods_df['Postal code'].apply(geolocator.query_postal_code)[["longitude"]]

In [149]:
Neighborhoods_df.head(11)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Part 3: Neighborhoods Clustering

In [154]:
#Drop Non Toronto Neighborhoods
toronto_neighborhoods_df = Neighborhoods_df[Neighborhoods_df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [155]:
#Geolocator to get Toronto latitude and longitude
geogy = gyNominatim(user_agent="Toronto_map")

In [156]:
#Get Toronto latitude and longitude
location = geogy.geocode('Toronto, ON')

In [157]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods_df['Latitude'], toronto_neighborhoods_df['Longitude'], toronto_neighborhoods_df['Borough'], toronto_neighborhoods_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [159]:
# The code was removed by Watson Studio for sharing.